In [1]:
import numpy as np
import copy
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras import regularizers
from keras.layers import Dropout
from sklearn.preprocessing import StandardScaler
from abc import ABC, abstractmethod
from random import random
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
import os
#initialize for random seeds/states
tf.keras.backend.clear_session()
os.environ['PYTHONHASHSEED'] = '0'
rng = np.random.default_rng(12345)
np.random.seed(42)
tf.random.set_seed(42)

In [2]:
rfloat = rng.random()

In [3]:
print(rfloat)

0.22733602246716966


In [4]:
class paper_game(ABC):
    def __init__(self, train, start_state, train_number=2000):
        self.state=start_state
        self.train = train
        self.train_number=train_number
        
    @abstractmethod
    def transition(self, state, action):
        #return next state
        pass
    
    @abstractmethod
    def reward(self, state, action):
        #return reward immediately after action
        pass
    
    @abstractmethod
    def policy(self, state):
        #return action
        pass
    
    @abstractmethod
    def Q(self, state, action):
        pass
    
    @abstractmethod
    def Q_update(self, alpha, gamma):
        pass

In [5]:
class tit_tac_toe(paper_game):
#state: 3x3 array to represent board config, with 1 representing self-checker, -1 representing opponent-checker, 0 representing empty
    def __init__(self, train, start_state=np.zeros((3,3), dtype=np.int8), train_number=2000):
        self.state=start_state
        self.status_func = win_status_function
        self.train = train
        self.train_number=train_number
    
    def transition(self, state, action):
        #action: a list of row and column index to indicate which cell is chosen
        new_state = copy.deepcopy(state)
        new_state[action[0]][action[1]] = 1
        return new_state
        #return next state
        
    def reward(self, state, action):
        s = transition(self, state, action)
        if 3 in np.sum(s, axis=0) or 3 in np.sum(s, axis=1) or 3 in s.diagonal or 3 in np.fliplr(s).diagonal():
            reward = 100
        return reward
        